In [1]:
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco

In [5]:
file = pd.ExcelFile('raw data/wuenic2021rev_web-update.xlsx')
sheets = file.sheet_names
sheets

['BCG',
 'DTP1',
 'DTP3',
 'HEPB3',
 'HEPBB',
 'HIB3',
 'IPV1',
 'MCV1',
 'MCV2',
 'PCV3',
 'POL3',
 'RCV1',
 'ROTAC',
 'YFV',
 'regional_global']

In [7]:
pd.read_excel('raw data/wuenic2021rev_web-update.xlsx', sheet_name=sheets[1])

,unicef_region,iso3,country,vaccine,2021,2020,2019,2018,2017,2016,...,1989,1988,1987,1986,1985,1984,1983,1982,1981,1980
0,ROSA,AFG,Afghanistan,DTP1,74,78,75,81,75,73,...,58.0,60.0,50.0,32.0,37.0,39.0,24.0,24.0,NaN,22.0
1,ECAR,ALB,Albania,DTP1,98,99,99,99,99,99,...,97.0,98.0,98.0,98.0,98.0,98.0,98.0,98.0,97.0,97.0
2,MENA,DZA,Algeria,DTP1,96,96,96,96,95,96,...,95.0,93.0,92.0,92.0,88.0,NaN,NaN,NaN,NaN,NaN
3,Non-programme,AND,Andorra,DTP1,99,99,99,99,99,99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ESAR,AGO,Angola,DTP1,57,61,67,74,68,74,...,41.0,33.0,31.0,31.0,28.0,28.0,25.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,LACR,VEN,Venezuela (Bolivarian Republic of),DTP1,73,73,85,84,95,97,...,80.0,79.0,80.0,80.0,73.0,58.0,80.0,77.0,77.0,79.0
191,EAPR,VNM,Viet Nam,DTP1,87,96,96,78,98,96,...,96.0,88.0,74.0,68.0,67.0,24.0,22.0,NaN,NaN,NaN
192,MENA,YEM,Yemen,DTP1,82,83,81,75,76,76,...,79.0,61.0,41.0,35.0,33.0,27.0,24.0,22.0,21.0,18.0
193,ESAR,ZMB,Zambia,DTP1,94,88,94,94,95,96,...,94.0,94.0,94.0,86.0,86.0,81.0,73.0,NaN,NaN,NaN
